### MaterialX Basics

**Importing MaterialX**

Test example to check that MaterialX modules are available and will run.

In [78]:
import MaterialX as mx

# Print the version of MaterialX
print('Hello MaterialX (Version %s)' % mx.__version__)

Hello MaterialX (Version 1.38.7)


**Loading Standard Libraries**

In [79]:
libraryPath = mx.FilePath('libraries')
# Check for library path on input args
stdlib = mx.createDocument()
searchPath = mx.FileSearchPath()
libFiles = mx.loadLibraries([ libraryPath ], searchPath, stdlib)

# Create main document and import the library document
doc = mx.createDocument()
doc.importLibrary(stdlib)

# Print out the names of the library files loaded
if libFiles:
    #for libFile in libFiles:
    print('Loaded %d library files.' % len(libFiles))
else:
    print('No libraries loaded.')

Loaded 41 library files.


### Nodes

**Node Creation**

There are a few options for creating a node:
1. By category and type.
2. By node definition name.
Please refer to the node library documentation for available categories, types and definitions.

In [80]:
# Creating Nodes: Method 1

# Method 1 : Create a node using category and type.
# - This method can be error prone as if the type is not specified then a string type is assumed.
# - An empty name implies a new unique name will be generated within scope the document. 
category = 'standard_surface'
name = ''
newNode = doc.addNode(category, name)
if newNode:
    print('Created erroneous node: (%s), path(%s), type(%s) ' % (newNode.getName(), newNode.getNamePath(), newNode.getType()))
    doc.removeNode(newNode.getName())
type = 'surfaceshader'
newNode = doc.addNode(category, name, type)
if newNode:
    print('Created correct node: (%s), path(%s), type(%s) ' % (newNode.getName(), newNode.getNamePath(), newNode.getType()))

# Method 1a: Supply a name
name = doc.createValidChildName(category)
newNode = doc.addNode(category, name, type)
if newNode:
    print('Created node with explicit name: (%s), path(%s):' % (newNode.getName(), newNode.getNamePath()))

# Validate
result = doc.validate()
if not result[0]:
    print('Invalid document. Errors: "%s"' % result[1])

Created erroneous node: (node1), path(node1), type(color3) 
Created correct node: (node1), path(node1), type(surfaceshader) 
Created node with explicit name: (standard_surface), path(standard_surface):


In [81]:
# Creating Nodes: Method 2

# Method 2: Use node definition name
# - This method requires knowledge of the actual nodedef name that easy to find.
shaderNode = None
shadername = doc.createValidChildName(category)
nodedef = doc.getNodeDef('ND_standard_surface_surfaceshader_100')
if nodedef:
    shaderNode = doc.addNodeInstance(nodedef, shadername)    
if newNode:
    shaderName = shaderNode.getName()
    print('Created node via nodedef name: (%s), path(%s):' % (shaderName, newNode.getNamePath()))



Created node via nodedef name: (standard_surface2), path(standard_surface):


In [82]:
# Method 2a: Search for appropriate nodedefs.
category = 'image'
nodedefs = doc.getMatchingNodeDefs(category)
desiredType = 'color3'
desiredNodedef = None
print('Scan for nodedef with category %s, type %s' % (category, desiredType))
for nd in nodedefs:
    if nd.getType() == desiredType:
        print('- Nodedef name: %s. Version: %s, Type: %s. Match: %d' % 
            (nd.getName(), nd.getVersionString(), nd.getType(), nd.getType() == desiredType))
        desiredNodedef = nd

imageNodePath = ""
if desiredNodedef:
    name = doc.createValidChildName(category)
    newNode = doc.addNodeInstance(desiredNodedef, name)
    if newNode:
        imageNodePath = newNode.getNamePath()
        print('- Created node by scanning nodedefs: (%s), type(%s):' % (newNode.getName(), newNode.getType()))
else:
    print('- Failed to find desirned nodedef.')

Scan for nodedef with category image, type color3
- Nodedef name: ND_image_color3. Version: , Type: color3. Match: 1
- Created node by scanning nodedefs: (image), type(color3):


**Finding Nodes**

Nodes can be found in a variety of different ways:
* By child name: If the name is know ahead of time this approach is suitable
* By path: A more robust way to find a child as the path explicity points to where in the element hierarchy the node resides. Element paths are always relative even if a "root" separator "/" is provided.
* By type. The singular version is generally not useful as the name itself is sufficient to return a unique element.

In [83]:
# Method 1: Get descendent using name.
shadernode = doc.getChild(shadername)
if shadernode:
    print('- Found node by name:', shadername)
shadernode = doc.getDescendant("/" + shadername)
if shadernode:
    print('- Found node by path:', shadername)
shadernode = doc.getChildOfType(mx.Node, shadername)
if shadernode:
    print('- Found node by type:', shadername)


- Found node by name: standard_surface2
- Found node by path: standard_surface2
- Found node by type: standard_surface2


Getting lists of nodes:

In [97]:
# Multiple element query
nodes = doc.getChildrenOfType(mx.Node)
if nodes:
    names = [ node.getName() for node in nodes ]
    print("- Found child nodes by type:", names)

nodes = doc.getNodes()
if nodes:
    names = [ node.getName() for node in nodes ]
    print("- Found child nodes by getNodes():", names)

nodes = doc.getChildren()
if nodes:
    # Just print out a few nodes as all children including definitions are in this list.
    names = [ nodes[i].getName() for i in range(0,10) ]
    print("- Found child nodes:", names)



- Found child nodes by type: ['node1', 'standard_surface', 'standard_surface2', 'image']
- Found child nodes by getNodes(): ['node1', 'standard_surface', 'standard_surface2', 'image']
- Found child nodes: ['ND_disney_brdf_2012_surface', 'IM_disney_brdf_2012_surface_brdf_explorer', 'ND_disney_bsdf_2015_surface', 'IM_disney_bsdf_2015_surface_pbrt', 'ND_gltf_pbr_surfaceshader', 'IMPL_gltf_pbr_surfaceshader', 'ND_gltf_colorimage', 'NG_gltf_colorimage', 'ND_gltf_image_color3_color3_1_0', 'NG_NG_gltf_image_color3_color3_1_0']


**Node Connections**

Connections are formed from a downstream input to an upstream output. For this a wrapper function is
added to hide some of the syntactic peculuarities. It will be added to as the types of connections
being considered is added.

One cumbersome thing is that a node instance when created as no inputs instantiated. So a check
must be made to see if it exists and if not added it. Then if input and outputs types match 
then the input can make the connection.

Additionally it is considered "invalid" to have both a value and a connection on an input, so
if a value has been set it must be removed.

Something like the 'connect()' method would be a useful utility to have in the core
API.

In [98]:
def connect(inputNode, inputName, outputNode, outputName):
    if not inputNode or not outputNode:
        return None

    inputPort = inputNode.addInputFromNodeDef(inputName)
    outputType = outputNode.getType()
    #if outputType == 'multioutput':
    outputPortFound = outputNode.getOutput(outputName)
    if outputPortFound:
        outputType = outputPortFound.getType()
    outputNodedef = outputNode.getNodeDef()
    if outputNodedef:
        outputPorts = outputNodedef.getOutputs()
        if outputPorts:                    
            if not outputName:
                outputType = outputPorts[0].getType()
                outputPortFound = outputPorts[0]
            else:
                for outputPort in outputPorts:
                    if outputPort.getName() == outputName:
                        outputPortFound = outputPort
                        outputType = outputPort.getType()
                        break                        

    if not outputPortFound:
        print('No output port found matching: ', outputName)        

    if inputPort.getType() != outputType:
        print('Input type (%s) and output type (%s) do not match: ' % (inputPort.getType(), outputType))
        return None

    if inputPort:
        inputPort.removeAttribute('value')
        inputPort.setNodeName(outputNode.getName())
        if outputName:
            inputPort.setOutput(outputName)
    return inputPort

# Connect image node to shader node previously created
imageNode = doc.getDescendant(imageNodePath)
shaderNode = doc.getChild(shadername)
if imageNode and shaderNode:
    inputConnnected = connect(shaderNode, "base_color", imageNode, "")
    if inputConnnected:
        print(inputConnnected)




No output port found matching:  aa


AttributeError: 'MaterialX.PyMaterialXCore.Input' object has no attribute 'setOutput'

**Node Deletion**

There is one option for removing nodes, and that is by name. It is not recommended to use low level
APIs such as 'removeChild()' as they can remove non-nodes.

Note that nodes are created under a graph container (`GraphElement`) and that both `Documents` and `Nodegraphs` are 
containers.

In [ ]:
# Deleting Nodes

# There is no way to remove all nodes at the same time, so we remove them
# one at a timme
print('Clear document contents:')
for node in doc.getNodes():
    print('- Remove node: %s. Type: %s. Version: %s. Nodedef: %s' %
     (node.getName(), node.getType(), node.getVersionString(), node.getNodeDefString()))
    doc.removeNode(node.getName())    

Clear document contents:
- Remove node: node1. Type: surfaceshader. Version: . Nodedef: 
- Remove node: standard_surface. Type: surfaceshader. Version: . Nodedef: 
- Remove node: standard_surface2. Type: surfaceshader. Version: . Nodedef: ND_standard_surface_surfaceshader_100
- Remove node: image. Type: color3. Version: . Nodedef: ND_image_color3
